In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import numpy as np
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup
from MPyDATA_examples.Olesik_et_al_2020.demo_plot_convergence  import plot
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot



In [3]:
GCs = np.linspace(.05,.95, 3)
nrs = np.linspace(32,128, 3, dtype = int)

In [4]:
from joblib import Parallel, parallel_backend, delayed

In [5]:
grid_layout = x_p2()
psi_coord = x_p2()
options = Options(n_iters=2,infinite_gauge=False, flux_corrected_transport=False,third_order_terms=False)
setup = Setup()

In [6]:
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, GMD

In [7]:
def analysis(nr, GC):
    setup = Setup(nr = nr, mixing_ratios_g_kg = [1,2])
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                lambda r: pdf_t(r * rh.units).magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n
    loc_of_maximum_num = simulation.r[np.argmax(numerical)]
    loc_of_maximum_anal = simulation.r[np.argmax(analytical)]
    maximum_num = np.max(numerical)
    maximum_anal = np.max(analytical)
#     mean = np.mean(numerical)
#     std = np.std(numerical)
    measure_location = loc_of_maximum_num / loc_of_maximum_anal
    measure_height = maximum_num / maximum_anal
    error_L2 = L2(numerical.magnitude, analytical.magnitude, simulation.out_steps[-1])
    error_GMD = np.log(GMD(numerical.magnitude, analytical.magnitude, t.magnitude)) / np.log(2)
    return nr, GC, error_GMD, measure_location, measure_height

In [8]:
with parallel_backend('threading', n_jobs=-2):
    results0 = Parallel(verbose=10)(
        delayed(analysis)(nr, GC)
        for nr in nrs
        for GC in GCs
    )

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   9 | elapsed:   44.1s remaining:  2.6min
[Parallel(n_jobs=-2)]: Done   3 out of   9 | elapsed:   51.0s remaining:  1.7min
[Parallel(n_jobs=-2)]: Done   4 out of   9 | elapsed:   57.8s remaining:  1.2min
[Parallel(n_jobs=-2)]: Done   5 out of   9 | elapsed:  1.1min remaining:   52.2s
[Parallel(n_jobs=-2)]: Done   6 out of   9 | elapsed:  1.2min remaining:   36.6s


ValueError: f(a) and f(b) must have different signs

In [ ]:
# results = np.array(results).T
results = [list(i) for i in zip(*results0)]
quality_metrics = {'location_ratio':results[3], 'height_ratio':results[4]}
results = np.array(results[0:3])

In [ ]:
plot(results[0], results[1], results[2])
# show_plot(filename = 'convergence.pdf')


In [ ]:
plot(results[0], results[1], quality_metrics['location_ratio'])
# show_plot(filename = 'convergence.pdf')


In [ ]:
plot(results[0], results[1], quality_metrics['height_ratio'])
show_plot(filename = 'convergence.pdf')